In [1]:
# pip install googletrans==4.0.0rc1

In [2]:
# pip install clean-text

In [3]:

# kurir -> courier 
# pengiriman -> delivery 
# melayani -> service 
# logistik -> logistics
# paket -> parcel
# tiba -> arrived
# cepat lambat


In [4]:
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import re # For regular expressions
from cleantext import clean


from googletrans import Translator

In [5]:
df = pd.read_csv('vinyl4.csv')

In [6]:
df.head()

,username,date,variation,rating,comment
0,r8os1vl7v3,2022-03-09 12:46:27,TOSCA,5,Daya serap:bgus\nHarga:murah\nBahan/Material:o...
1,zahrazaskiakhoirunisahy,2022-03-13 07:58:32,HIJAU MUDA,4,Harga:murah bgt\nBahan/Material:lentur\nDaya s...
2,h*****7,2022-02-08 23:01:05,ABU,5,Harga:muraah\n\nAlhamdulillah pengiriman nya c...
3,yebitramudia,2022-01-21 19:22:29,TOSCA,5,Harga:termurah\nBahan/Material:--\nDaya serap:...
4,cukonndute,2021-12-19 23:04:48,KUNING,5,Harga:sangat murahhhhhhhhhhhhhhhhhhhhhhhhhhh\n...


In [7]:
def clean_text_round1(text):
    '''Make text lowercase, remove punctuation, remove words containing numbers and lemmatize/stem.'''
    text = str(text).lower()
    text = re.sub('#\w*', '', text)
    text = re.sub('@\w*', '', text)    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = clean(text, no_emoji=True)
    return text

# Let's take a look at the updated text
df['clean'] = pd.DataFrame(df['comment'].apply(clean_text_round1))


In [8]:
df['Reviews with the words courier, delivery etc'] = df['clean'].str.replace('\n',' ')

In [9]:
# df1 = df[df['clean'].str.contains("pengiriman","kurir")]
bow_list = ['kurir', 'pengiriman', 'melayani', 'logistik', 'paket', 'tiba']

In [10]:
df1 = df[df['Reviews with the words courier, delivery etc'].str.contains('|'.join(bow_list))]
len(df1.index)

230

In [11]:
df2 = df1[df1['Reviews with the words courier, delivery etc'].str.contains("cepat")]

In [12]:
df2 = df2.rename(columns={'clean': 'Reviews with the word fast'})
len(df2.index)

126

In [13]:
df3 = df1[df1['Reviews with the words courier, delivery etc'].str.contains("lambat")]

In [14]:
df3= df3.rename(columns={'clean': 'Reviews with the word slow'})
len(df3.index)

1

In [15]:
frames = [df1['Reviews with the words courier, delivery etc'], df2['Reviews with the word fast'], df3['Reviews with the word slow']]
result = pd.concat(frames, axis =1)

In [16]:
result

,"Reviews with the words courier, delivery etc",Reviews with the word fast,Reviews with the word slow
2,hargamuraah alhamdulillah pengiriman nya cepat...,hargamuraah\nalhamdulillah pengiriman nya cepa...,NaN
3,hargatermurah bahanmaterial daya serapcepat ak...,hargatermurah\nbahanmaterial\ndaya serapcepat\...,NaN
4,hargasangat murahhhhhhhhhhhhhhhhhhhhhhhhhhh ba...,hargasangat murahhhhhhhhhhhhhhhhhhhhhhhhhhh\nb...,NaN
7,hargamurah daya serapbagus bahanmaterialpvc al...,NaN,NaN
8,hargamurmer daya serapwaterproof bahanmaterial...,hargamurmer\ndaya serapwaterproof\nbahanmateri...,NaN
...,...,...,...
1234,pengiriman lama,NaN,NaN
1238,kualitas barang bagus pengiriman cepat harga t...,kualitas barang bagus pengiriman cepat harga t...,NaN
1244,hargamayan daya serapbaik bahanmaterialbagus s...,NaN,NaN
1250,barang bagus pengiriman lambat,NaN,barang bagus\npengiriman lambat


In [17]:
result.to_csv('vinyl4.csv')

In [18]:
# #initialize the Translator
# translator = Translator()
# source_lan = "id"  #en is the code for Hindi Language
# translated_to= "en" #hi is the code for Hindi Language

In [19]:
# result['translated_review (fast)'] = result['Reviews with the word fast'].apply(lambda row: translator.translate(row, src=source_lan, dest = translated_to).text)

In [20]:
# result['translated_review (slow)'] = result['Reviews with the word slow'].apply(lambda row: translator.translate(row, src=source_lan, dest = translated_to).text)

In [21]:
result['Reviews with the word slow'].dropna()

1250    barang bagus\npengiriman lambat
Name: Reviews with the word slow, dtype: object

In [22]:
result['Reviews with the word slow'].dropna().to_csv('vinyl4_slow.csv')